In [ ]:
from tanakinator.models import init, db

In [ ]:
init()

In [ ]:
from tanakinator import models

In [ ]:
solution_names = ["英語", "Python", "C++", "日本語"]
question_messages = [
    "プログラミング言語ですか？",
    "インタプリタ型言語ですか？",
    "世界で広く使われていますか？"
]
feature_matrix = [
    [-1.0, -1.0,  1.0],
    [ 1.0,  1.0,  1,0],
    [ 1.0, -1.0,  1.0],
    [-1.0, -1.0, -1.0]
]

In [ ]:
try:
    for s_name in solution_names:
        s = models.Solution()
        s.name = s_name
        db.session.add(s)
        db.session.commit()

    for q_message in question_messages:
        q = models.Question()
        q.message = q_message
        db.session.add(q)
        db.session.commit()
except:
    print("rejected.")
    db.session.rollback()

In [ ]:
for s in models.Solution.query.all():
    for q in models.Question.query.all():
        s_idx = solution_names.index(s.name)
        q_idx = question_messages.index(q.message)
        value = feature_matrix[s_idx][q_idx]

        f = models.Feature()
        f.solution = s
        f.question = q
        f.value = value
        try:
            db.session.add(f)
            db.session.commit()
        except:
            print("rejected.")
            db.session.rollback()

In [ ]:
for s in models.Solution.query.all():
    print(f"name: {s.name}")
    for f in s.features:
        question_message = f.question.message
        answer = 'はい．' if f.value >= 0.0 else 'いいえ．'
        print(f"\tQ.{question_message}")
        print(f"\tA.{answer}\n")

In [ ]:
answers = []

answer_values = []
for q in question_messages:
    ans = input(q + "[y/n]")
    if ans != 'y' and ans != 'n':
        raise RuntimeError("Invalid answer.")
    ans = 1.0 if ans == 'y' else -1.0
    answer_values.append(float(ans))

for q_msg, ans in zip(question_messages, answer_values):
    a  = models.Answer()
    q = db.session.query(models.Question).filter_by(message=q_msg).first()
    a.question = q
    a.value = ans
    answers.append(a)

In [ ]:
score_table = {s.id: 0.0 for s in models.Solution.query.all()}
for ans in answers:
    q_features = db.session.query(models.Feature).filter_by(question_id=ans.question.id)
    for f in q_features:
        score = ans.value * f.value
        score_table[f.solution.id] += score

print(score_table)
most_likely_solution = models.Solution.query.get(max(score_table, key=score_table.get))
print(most_likely_solution.name)